***IMPORTS Y ACCESO A DATOS DE CSV BICIS***

In [437]:
#Importación de paquetes y lectura de csv:
from sklearn.linear_model import Ridge
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
import pandas as pd
import numpy as np
import random
from sklearn import model_selection
from scipy.spatial.distance import euclidean
from scipy.stats import spearmanr
import xgboost as xgb
from xgboost import DMatrix
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
import copy

random.seed(27)
dataBicis = pd.read_csv('hour.csv')
dataBicis = dataBicis.drop("dteday", axis=1)
dataBicis = dataBicis.drop("instant", axis=1)
atributosBicis = dataBicis.loc[:, 'season':'registered']
objetivoBicis = dataBicis['cnt']

(atributos_entrenamientoBicis, atributos_pruebaBicis,
 objetivo_entrenamientoBicis, objetivo_pruebaBicis) = model_selection.train_test_split(
    atributosBicis, objetivoBicis, test_size=.33)

atributos_entrenamientoBicis = atributos_entrenamientoBicis.values
objetivo_entrenamientoBicis = objetivo_entrenamientoBicis.values
atributos_pruebaBicis = atributos_pruebaBicis.values
objetivo_pruebaBicis= objetivo_pruebaBicis.values

rangosBicis = {} #Diccionario con los valores máximo y mínimo de cada atributo

for columna in dataBicis.columns:
    valor_min = dataBicis[columna].min()
    valor_max = dataBicis[columna].max()
    rangosBicis[columna] = {'min': valor_min, 'max': valor_max}

***MÉTODO LIME***

In [438]:
def metodo_lime(N,x,k,model,rangos):
    random.seed(1)
    Xprima=[]
    R=[]
    W=[]
    Yprima= []
    for i in range (0,N):
        atributosAPerturbar=seleccionar_atributos_aleatorios(x,k)
        xprima = perturbar_muestra(x,atributosAPerturbar,rangos)
        w = distancia_entre_muestras(x,xprima)
        r=atributosAPerturbar
        Xprima.append(xprima)
        R.append(r)
        W.append(w)
        yprima = prediccion_de_modelo(xprima,model)
        Yprima.append(yprima)
    W=np.array(W)
    W = W.reshape(W.shape[0])
    G = entrenar_modelo_ridge(R,Yprima,W)
    return G.coef_

#Se seleccional los k atributos aleatorios a modificar

def seleccionar_atributos_aleatorios(dataframe, k):
    columnas = dataframe.columns.tolist()
    seleccionados = random.sample(range(len(columnas)), k)
    lista = [0] * len(columnas)
    for i in seleccionados:
        lista[i] = 1
    #print(lista)
    return lista

#Se perturba la muestra original de forma que los atributos seleccionados pueden variar en un rango en función del valor máximo y mínimo de dicho atributo
def perturbar_muestra(x, atributosAPerturbar,rangos):
    xprima = x.copy()
    for i in range(0,len(atributosAPerturbar)):
        if atributosAPerturbar[i] == 1:
            rango_min = xprima[i] - rangos[list(rangos.keys())[i]]['min']
            rango_max = rangos[list(rangos.keys())[i]]['max'] - xprima[i]
            xprima[i] = random.uniform(rango_min, rango_max)
    return xprima

# Se calcula la distancia coseno entre la muestra original y la perturbada
def distancia_entre_muestras(x,xprima):
    return cosine_distances(x.values, xprima.values)

#Se predice el resultado de la muestra con el modelo elegido
def prediccion_de_modelo(xprima,model):
    if not isinstance(model, (RandomForestRegressor, RandomForestClassifier)):
        xprima = DMatrix(xprima)
    y_pred = model.predict(xprima)
    return y_pred


#Se entrena el modelo de regresión Ridge con las muestras perturbadas y los pesos calculados
def entrenar_modelo_ridge(R, Yprima, W):
    modelo_entrenado = Ridge(alpha=1.0) 
    modelo_entrenado.fit(R, Yprima, sample_weight=W)
    return modelo_entrenado

***MODELOS BICIS***

In [439]:
#RandomForest
#Creamos el modelo RandomForest (En este caso es Regressor ya que se trata de una tarea de regresión)
modelRandomForestBicis = RandomForestRegressor(n_estimators=100)
#Entrenamos el modelo con los atributos y objetivo de entrenamiento
modelRandomForestBicis.fit(atributos_entrenamientoBicis, objetivo_entrenamientoBicis)

#XGBoost
# Creamos el objeto DMatrix para XGBoost
dtrainBicis = xgb.DMatrix(atributos_entrenamientoBicis, label=objetivo_entrenamientoBicis)
dtestBicis = xgb.DMatrix(atributos_pruebaBicis)
dtestBicis2 = xgb.DMatrix(atributos_pruebaBicis)
# Definimos los parámetros del modelo XGBoost
paramsBicis = {
    'objective': 'reg:squarederror',  # Objetivo para regresión
    'eval_metric': 'rmse'  # Métrica de evaluación, error cuadrático medio (RMSE)
}

# Entrenamos el modelo XGBoost
modelXGBoostBicis = xgb.train(paramsBicis, dtrainBicis)

In [440]:
clon_model_XGboost_bicis = copy.deepcopy(modelXGBoostBicis)
prediccionesBicisXG = clon_model_XGboost_bicis.predict(dtestBicis)
explicacionesBicisXG = []
atributosAcambiar = 5
for i in range(0,256):  
    nuevos_ejemplos2 = pd.DataFrame(atributos_pruebaBicis).iloc[i]
    dataframe = pd.DataFrame([nuevos_ejemplos2])
    explicacion = metodo_lime(10, dataframe, atributosAcambiar, clon_model_XGboost_bicis,rangosBicis)
    explicacionesBicisXG.append(explicacion)
print(explicacionesBicisXG)

[array([[-19.06200839, -28.1487068 ,  -9.55311723,  -9.00597514,
         -1.87318867, -76.9388543 ,  43.79270575,  42.69926577,
        -10.17088594,  -5.85902219, -26.27551813,   4.36494477,
          9.15927772,  86.87108278]]), array([[ -41.75719567,  -40.9901878 ,  -12.7447944 ,  -36.93193232,
          -2.85372133, -112.84375866,   70.20890441,   55.37964865,
          -0.61924727,  -42.08203071,  -38.13646647,   39.40382385,
          38.2028902 ,  125.76406753]]), array([[-2.04024809e+00, -2.99441735e-01, -4.28060271e-03,
        -2.02925496e+00, -1.53293131e-03, -1.15235282e+00,
         2.23287005e+00, -1.07623663e+00, -6.32197079e-01,
        -2.12777282e+00, -2.97908804e-01,  2.12978348e+00,
         4.13839594e+00,  1.16017700e+00]]), array([[ 1.60614725e+00, -1.11863244e+01, -3.21081337e-01,
        -1.28110560e+00, -1.62186218e-02, -1.95597036e+01,
         1.02888774e+01,  9.59190754e+00,  7.58981479e-01,
        -2.48118013e-01, -1.11701057e+01,  2.49720977e-01,
      

In [441]:
prediccionesBicisRF = modelRandomForestBicis.predict(atributos_pruebaBicis)
explicacionesBicisRF = []
for i in range(0,256):  
    nuevos_ejemplos2 = pd.DataFrame(atributos_pruebaBicis).iloc[i]
    dataframe = pd.DataFrame([nuevos_ejemplos2])
    explicacion = metodo_lime(10, dataframe, atributosAcambiar, modelRandomForestBicis,rangosBicis)
    explicacionesBicisRF.append(explicacion)
print(explicacionesBicisRF)

[array([[-18.3881594 , -29.16985712,  -9.89555481,  -9.41480476,
         -1.99793835, -79.51029316,  48.83258705,  40.57326091,
        -11.99993442,  -1.41114263, -27.17191877,  -0.17208897,
          9.90528971,  89.82055472]]), array([[ -42.73967735,  -42.71029059,  -12.92467551,  -37.46869559,
          -2.97038272, -116.64455038,   77.41095725,   52.15826864,
          -1.29523631,  -38.1478238 ,  -39.73990787,   35.36542728,
          39.94937485,  129.75721209]]), array([[-2.49383845e+00, -3.11545333e-01, -4.60255757e-03,
        -2.23626727e+00, -1.65505149e-03, -1.21995814e+00,
         2.73303232e+00, -1.50847163e+00, -3.93128827e-01,
        -2.29971898e+00, -3.09890281e-01,  2.30186758e+00,
         4.51581228e+00,  1.22836434e+00]]), array([[-2.06116824e-01, -1.04124409e+01, -3.40463347e-01,
        -1.26320819e+00, -1.73890119e-02, -1.94010425e+01,
         1.06196921e+01,  9.12181370e+00,  1.11285132e+00,
        -9.18240301e-01, -1.03950519e+01,  9.19925391e-01,
      

***VALIDACIÓN DE MODELO***

In [442]:
def evaluate_model(model, Xentrenamiento, yentrenamiento, Xtest, ytest):
    # Validación cruzada
    cv_scores = cross_val_score(model, Xentrenamiento, yentrenamiento, cv=2)
    mean_cv_score = np.mean(cv_scores)
    
    print("Puntuación media de validación cruzada:", mean_cv_score)
    
    #.score
    score = model.score(Xtest, ytest)
    print("Puntuación en el conjunto de prueba:", score)

In [443]:
evaluate_model(modelRandomForestBicis, atributos_entrenamientoBicis, objetivo_entrenamientoBicis, atributos_pruebaBicis, objetivo_pruebaBicis)

Puntuación media de validación cruzada: 0.9995360953589163
Puntuación en el conjunto de prueba: 0.999739620426297


******

In [444]:


nuevos_ejemplos2 = pd.DataFrame(atributos_entrenamientoBicis).iloc[3]
dataframe = pd.DataFrame([nuevos_ejemplos2])
print(dataframe)


Iterac=10
atributosAcambiar=5

nuevos_ejemplos = pd.DataFrame([[1,0,1,8,0,6,0,1,0.24,0.2879,0.75,0,1,7]])
#print("Prediccion RF",prediccion_de_modelo(dataframe,modelRandomForestBicis))
#print("Prediccion XG",prediccion_de_modelo(dataframe,modelXGBoostBicis))
#print("Método LIME con Random Forest: ",metodo_lime(Iterac,dataframe,atributosAcambiar,modelRandomForestBicis,rangosBicis))

#print("Método LIME con XGBoost: ",metodo_lime(Iterac,dataframe,atributosAcambiar,modelXGBoostBicis,rangosBicis))

    0    1     2    3    4    5    6    7     8       9     10      11    12  \
3  4.0  1.0  10.0  6.0  0.0  3.0  1.0  1.0  0.48  0.4697  0.82  0.1045  11.0   

      13  
3  162.0  


******

***

***ACCESO A DATOS DE CSV PEN***

In [445]:

# Cargar el archivo CSV
pen = pd.read_csv('pendigits.csv', header=0, names=['a1','a2','a3','a4','a5','a6','a7','a8','a9','a10','a11','a12','a13','a14','a15','a16',"class-objective"])

atributosPen = pen.loc[:, 'a1':'a16']
objetivoPen = pen['class-objective']

rangosPen = {} #Diccionario con los valores máximo y mínimo de cada atributo

for columna in pen.columns:
    valor_min = pen[columna].min()
    valor_max = pen[columna].max()
    rangosPen[columna] = {'min': valor_min, 'max': valor_max}
    


In [446]:
(atributos_entrenamientoPen, atributos_pruebaPen,
 objetivo_entrenamientoPen, objetivo_pruebaPen) = model_selection.train_test_split(
    atributosPen, objetivoPen, test_size=.33,stratify=objetivoPen)

atributos_entrenamientoPen = atributos_entrenamientoPen.values
objetivo_entrenamientoPen = objetivo_entrenamientoPen.values
atributos_pruebaPen = atributos_pruebaPen.values
objetivo_pruebaPen= objetivo_pruebaPen.values

print(atributos_entrenamientoPen)

[[  9  79  81 ...   0  15   3]
 [100  89  39 ...  24  23   0]
 [ 41  81  48 ...   0 100  31]
 ...
 [  0  92  42 ...  22 100  21]
 [100  92  79 ...   0   0   1]
 [  0  80  24 ...  36 100  38]]


***MODELOS PEN***

In [447]:

modelRandomForestPen = RandomForestClassifier(n_estimators=100)
#Entrenamos el modelo con los atributos y objetivo de entrenamiento
modelRandomForestPen.fit(atributos_entrenamientoPen, objetivo_entrenamientoPen)

#XGBoost
# Creamos el objeto DMatrix para XGBoost
dtrainPen = xgb.DMatrix(atributos_entrenamientoPen, label=objetivo_entrenamientoPen)
dtestPen = xgb.DMatrix(atributos_pruebaPen)
dtestPen2 = xgb.DMatrix(atributos_pruebaPen)

# Definimos los parámetros del modelo XGBoost
paramsPen = {
    'objective': 'multi:softmax',  # Función objetivo para clasificación multiclase
    'num_class': 10,       # Número de clases en tu problema
    'eval_metric': 'merror' 
}

# Entrenamos el modelo XGBoost
modelXGBoostPen = xgb.train(paramsPen, dtrainPen)

In [448]:
clon_model_XGboost_pen = copy.deepcopy(modelXGBoostPen)
prediccionesPenXG = clon_model_XGboost_pen.predict(dtestPen)
explicacionesPenXG = []
for i in range(0,256):  
    nuevos_ejemplos2 = pd.DataFrame(atributos_pruebaPen).iloc[i]
    dataframe = pd.DataFrame([nuevos_ejemplos2])
    explicacion = metodo_lime(10, dataframe, atributosAcambiar, clon_model_XGboost_pen, rangosPen)
    explicacionesPenXG.append(explicacion)
print(explicacionesPenXG)

[array([[-0.28382758, -0.08530143, -0.04191254, -0.17780256, -0.13051287,
        -0.13263615, -0.12909574, -0.06906526,  0.18427671, -0.16093407,
         0.44476164,  0.28621974, -0.24081849,  0.24264104,  0.42618198,
        -0.13217442]]), array([[-0.39843831, -0.17668353,  0.02461587, -0.37561091, -0.2062966 ,
         0.04195622,  0.22659764,  0.03152508,  0.13460376,  0.05478775,
         0.34365056, -0.08508467, -0.16812109, -0.00356525,  0.35810951,
         0.19795396]]), array([[ 0.05410602, -0.19147007,  0.25564688,  0.10093516,  0.08192338,
        -0.33553732, -0.1554323 , -0.4772876 ,  0.93971927, -0.46323082,
         0.4091248 ,  0.3622171 , -0.77004555,  0.04953515,  0.24188292,
        -0.10208703]]), array([[-0.23364692,  0.09779887, -0.03242126, -0.14584635,  0.02034441,
        -0.04657873,  0.08394031, -0.0631303 , -0.03085721,  0.31197784,
        -0.07833092, -0.14550657, -0.01453647,  0.22221107, -0.11420306,
         0.16878528]]), array([[ 0.06819978, -0.011

In [449]:
prediccionesPenRF = modelRandomForestPen.predict(atributos_pruebaPen)
explicacionesPenRF = []
for i in range(0,256):  
    nuevos_ejemplos2 = pd.DataFrame(atributos_pruebaPen).iloc[i]
    dataframe = pd.DataFrame([nuevos_ejemplos2])
    explicacion = metodo_lime(10, dataframe, atributosAcambiar, modelRandomForestPen, rangosPen)
    explicacionesPenRF.append(explicacion)
print(explicacionesPenRF)

[array([[-0.16237954,  0.19724779,  0.12223139,  0.03824393,  0.27455573,
        -0.30003685, -0.27827975, -0.35045975,  0.2975488 , -0.08581156,
         0.24819109,  0.10313741, -0.22396562,  0.22167408,  0.10757901,
        -0.20947616]]), array([[-0.08845292, -0.10022879, -0.16553873,  0.05246852, -0.20471   ,
        -0.25740188, -0.04827613, -0.11981127,  0.27546298,  0.03934734,
         0.04910558,  0.36946657, -0.36642245,  0.53621361, -0.0198391 ,
         0.04861666]]), array([[ 0.05410602, -0.19147007,  0.25564688,  0.10093516,  0.08192338,
        -0.33553732, -0.1554323 , -0.4772876 ,  0.93971927, -0.46323082,
         0.4091248 ,  0.3622171 , -0.77004555,  0.04953515,  0.24188292,
        -0.10208703]]), array([[-0.17731812, -0.02356465, -0.02714646, -0.11140325, -0.08378043,
        -0.03452279,  0.14020769, -0.04224031,  0.06748221,  0.21340005,
        -0.03608193, -0.08781934, -0.07637564,  0.14017323, -0.08908152,
         0.22807126]]), array([[ 0.08289538, -0.012

In [450]:
nuevos_ejemplos2 = pd.DataFrame(atributos_entrenamientoPen).iloc[2]
dataframe = pd.DataFrame([nuevos_ejemplos2])
print(dataframe)
Iterac=10
atributosAcambiar=5


#print("Prediccion RF",prediccion_de_modelo(dataframe,modelRandomForestPen))
#print("Prediccion XG",prediccion_de_modelo(dataframe,modelXGBoostPen))
#print("Método LIME con Random Forest: ",metodo_lime(Iterac,dataframe,atributosAcambiar,modelRandomForestPen,rangosPen))
#print("Método LIME con XGBoost: ",metodo_lime(Iterac,dataframe,atributosAcambiar,modelXGBoostPen,rangosPen))

   0   1   2    3   4   5   6   7   8   9   10  11  12  13   14  15
2  41  81  48  100  67  71  50  36   0  26  29  19  72   0  100  31


***VALIDACIÓN***

In [451]:
evaluate_model(modelRandomForestPen, atributos_entrenamientoPen, objetivo_entrenamientoPen, atributos_pruebaPen, objetivo_pruebaPen)

Puntuación media de validación cruzada: 0.9816733067729084
Puntuación en el conjunto de prueba: 0.9911039223615042


In [452]:
explicaciones_alteradas_RF_Bicis=[]
explicaciones_alteradas_RF_Pen=[]
explicaciones_alteradas_XG_Bicis=[]
explicaciones_alteradas_XG_Pen=[]

def eliminar_irrelevantes_RF(pesos, k, muestra, modelo,rangos):
    array = np.array(pesos).flatten()
    pesos_abs = np.abs(array)
    indices = np.argsort(pesos_abs)[:k]
    copia = muestra.copy()
    for i in range(len(muestra)):
        if i in indices:
            copia[i] = 0
    muestra_2d = copia.reshape(1, -1)
    prediccion_nueva = modelo.predict(muestra_2d)
    dataframe = pd.DataFrame(muestra_2d)
    explicacion = metodo_lime(10, dataframe, atributosAcambiar, modelo, rangos)
    if rangos == rangosBicis:
        explicaciones_alteradas_RF_Bicis.append(explicacion)
    else:
        explicaciones_alteradas_RF_Pen.append(explicacion)
    return prediccion_nueva



def eliminar_irrelevantes_XG(pesos, k, muestra, modelo,rangos):
    clon_model = copy.deepcopy(modelo)
    array = np.array(pesos).flatten()
    pesos_abs = np.abs(array)
    indices = np.argsort(pesos_abs)[:k]
    
    data = muestra.get_data()
    data_copy = data.copy()
    label = muestra.get_label()
    weight = muestra.get_weight()
    base_margin = muestra.get_base_margin()
    feature_names = muestra.feature_names
    # Copiar los valores y crear una nueva muestra
    
    for i in indices:
        data_copy[0,i]=0
    copia = xgb.DMatrix(data_copy, label=label, weight=weight, base_margin=base_margin)
    dataframe = pd.DataFrame([copia.get_data().data], columns=feature_names)
    prediccion_nueva = clon_model.predict(copia)
    explicacion = metodo_lime(10, dataframe, atributosAcambiar, clon_model, rangos)
    if rangos == rangosBicis:
        explicaciones_alteradas_XG_Bicis.append(explicacion)
    else:
        explicaciones_alteradas_XG_Pen.append(explicacion)
    return prediccion_nueva




In [453]:
predicciones_RF_Bicis = prediccionesBicisRF[:256]
objetivos_bicis = objetivo_pruebaBicis[:256]
explicaciones_RF_Bicis = explicacionesBicisRF

predicciones_XG_Bicis = prediccionesBicisXG[:256]
explicaciones_XG_Bicis = explicacionesBicisXG

predicciones_RF_Pen = prediccionesPenRF[:256]
objetivos_pen = objetivo_pruebaPen[:256]
explicaciones_RF_Pen = explicacionesPenRF

predicciones_XG_Pen = prediccionesPenXG[:256]
explicaciones_XG_Pen = explicacionesPenXG

In [454]:
dtestBicis = xgb.DMatrix(atributos_pruebaBicis)
dtestPen = xgb.DMatrix(atributos_pruebaPen)
predicciones_alteradas_RF_Bicis=[]
predicciones_alteradas_RF_Pen=[]
predicciones_alteradas_XG_Bicis=[]
predicciones_alteradas_XG_Pen=[]

for i in range(256):
    
    pred = eliminar_irrelevantes_RF(explicaciones_RF_Bicis[i],5,atributos_pruebaBicis[i],modelRandomForestBicis,rangosBicis)
    predicciones_alteradas_RF_Bicis.append(pred)
    pred = eliminar_irrelevantes_RF(explicaciones_RF_Pen[i],5,atributos_pruebaPen[i],modelRandomForestPen,rangosPen)
    predicciones_alteradas_RF_Pen.append(pred)
    pred = eliminar_irrelevantes_XG(explicaciones_XG_Pen[i],5,dtestPen.slice([i]),modelXGBoostPen,rangosPen)
    predicciones_alteradas_XG_Pen.append(pred)
    pred = eliminar_irrelevantes_XG(explicaciones_XG_Bicis[i],5,dtestBicis.slice([i]),modelXGBoostBicis,rangosBicis)
    predicciones_alteradas_XG_Bicis.append(pred)
    
print(len(predicciones_alteradas_XG_Bicis))
print(len(predicciones_alteradas_XG_Pen))

256
256


***MÉTRICAS***

In [538]:
# Identidad

def identidad(i, muestra, lista_explicaciones):
    e = lista_explicaciones[i]
    return np.linalg.norm(muestra - muestra) != 0 or np.linalg.norm(e - e) == 0

# Separabilidad

def separabilidad(i,j,muestra_i,muestra_j, lista_explicaciones):
    e1 = lista_explicaciones[i]
    e2 = lista_explicaciones[j]
    return np.linalg.norm(muestra_i - muestra_j) == 0 or np.linalg.norm(e1 - e2) > 0 

# Estabilidad

def estabilidad(i, muestra, lista_muestras, lista_explicaciones):
    e = lista_explicaciones[i]
    d_muestras = np.array([np.linalg.norm(muestra - x) for x in lista_muestras])
    d_explicaciones = np.array([np.linalg.norm(e - x) for x in lista_explicaciones])
    return spearmanr(d_muestras, d_explicaciones)[0] > 0

# Selectividad

def selectividad(muestras, n, modelo, lista_objetivos, tipo):
    lista_auc = []
    for i in range(n):
        # segun el tipo de tarea el valor AUC se calculará de una forma u otra
        if tipo == "regresion":
            muestras_actualizadas = np.copy(muestras)
            # se asignan a 0 las i muestras
            muestras_actualizadas[:, i] = 0
            predicciones = modelo.predict(muestras_actualizadas)
            # AUC se calcula a través del error cuadrático medio
            auc = mean_squared_error(predicciones[:256], lista_objetivos)
            lista_auc.append(auc)
        elif tipo == "clasificacion":
            muestras_actualizadas = np.copy(muestras)
            # se asignan a 0 las i muestras
            muestras_actualizadas[:, i] = 0
            predicciones = modelo.predict_proba(muestras_actualizadas)
            auc = roc_auc_score(lista_objetivos, predicciones[:256], multi_class='ovr')
            lista_auc.append(auc)
        elif tipo == "xgboost":
            # datos de la matriz
            data = muestras.get_data()
            data_copy = data.copy()
            label = muestras.get_label()
            weight = muestras.get_weight()
            base_margin = muestras.get_base_margin()
            # se asignan a 0 las i muestras
            data_copy[0,i]=0
            # con los datos de la matriz creo una copia y extraigo sus predicciones
            copia = xgb.DMatrix(data_copy, label=label, weight=weight, base_margin=base_margin)
            predicciones = modelo.predict(copia)
            # AUC se calcula a través del error cuadrático medio
            auc = mean_squared_error(predicciones[:256], lista_objetivos)
            lista_auc.append(auc)
    # el valor final de AUC es la media de todos los AUC obtenidos
    return np.mean(lista_auc)

# Coherencia
def coherencia(i, lista_predicciones, lista_objetivo,predicciones_actualizadas):
    error_prediccion_original = lista_predicciones[i] - lista_objetivo[i]
    error_prediccion_nuevo = predicciones_actualizadas[i] - lista_objetivo[i]
    return np.abs(error_prediccion_original - error_prediccion_nuevo)

# Completitud

def completitud(lista_predicciones, lista_objetivo,explicaciones,explicaciones_actualizadas):
    error_prediccion = np.mean(lista_predicciones - lista_objetivo)
    error_explicacion = np.mean([explicaciones[i] - explicaciones_actualizadas[i] for i in range(256)])
    return error_explicacion / error_prediccion

# Congruencia
def congruencia(lista_predicciones, lista_objetivo,predicciones_actualizadas):
    sumatorio_coherencia = 0
    sumatorio_desviacion = 0
    # cálculo de la coherencia promedio
    for i in range(len(lista_predicciones)):
        alpha = coherencia(i,lista_predicciones, lista_objetivo,predicciones_actualizadas)
        sumatorio_coherencia += alpha
    coherencia_promedio = sumatorio_coherencia / len(lista_predicciones)
    # cálculo del radicando de la raíz cuadrada
    for i in range(len(lista_predicciones)):
        alpha = coherencia(i,lista_predicciones, lista_objetivo,predicciones_actualizadas)
        sumatorio_desviacion += np.power(alpha-coherencia_promedio,2)
    arg = sumatorio_desviacion / len(lista_predicciones)
    return np.sqrt(arg)

In [539]:
# Bicis Random Forest

# Identidad

res_identidad = True
for i in range(0, 256):
    res_identidad &= identidad(i,atributos_pruebaBicis[i],explicaciones_RF_Bicis)
print("\nIdentidad:", res_identidad)

# Separabilidad

res_separabilidad = True
for i in range(0, 255):
    res_separabilidad &= separabilidad(i, 255,atributos_pruebaBicis[i],atributos_pruebaBicis[255],explicaciones_RF_Bicis)
print("\nSeparabilidad:", res_separabilidad)

# Estabilidad

res_estabilidad = True
for i in range(0, 256):
    res_estabilidad &= estabilidad(i,atributos_pruebaBicis[i],atributos_pruebaBicis[:256],explicaciones_RF_Bicis)
print("\nEstabilidad:", res_estabilidad)

# Selectividad

print("\nSelectividad:", selectividad(atributos_pruebaBicis,5,modelRandomForestBicis,objetivos_bicis,"regresion"))

# Coherencia

res_coherencia = 0
for i in range(0, 256):
    res_coherencia += coherencia(i,predicciones_RF_Bicis,objetivos_bicis,predicciones_alteradas_RF_Bicis)[0]
print("\nCoherencia:", res_coherencia/256)
    
# Completitud

res_completitud = completitud(predicciones_RF_Bicis,objetivos_bicis,explicaciones_RF_Bicis,explicaciones_alteradas_RF_Bicis)
print("\nCompletitud:", res_completitud)

# Congruencia

print ("\nCongruencia:", congruencia(predicciones_RF_Bicis, objetivos_bicis,predicciones_alteradas_RF_Bicis)[0])


Identidad: True

Separabilidad: True

Estabilidad: False

Selectividad: 7.837763906250002

Coherencia: 17.084140625000014

Completitud: 7.747052315958311e-16

Congruencia: 43.96637379022148


In [540]:
# Bicis XGBoost

# Identidad

res_identidad = True
for i in range(0, 256):
    res_identidad &= identidad(i,dtestBicis.slice([i]).get_data().data,explicaciones_XG_Bicis)
print("\nIdentidad:", res_identidad)

# Separabilidad

res_separabilidad = True
for i in range(0, 255):
    res_separabilidad &= separabilidad(i, 255,dtestBicis.slice([i]).get_data().data,dtestBicis.slice([255]).get_data().data,explicaciones_XG_Bicis)
print("\nSeparabilidad:", res_separabilidad)

# Selectividad

print("\nSelectividad:", selectividad(dtestBicis,5,modelXGBoostBicis,objetivos_bicis,"xgboost"))

# Coherencia

res_coherencia = 0
for i in range(0, 256):
    res_coherencia += coherencia(i,predicciones_XG_Bicis,objetivos_bicis,predicciones_alteradas_XG_Bicis)[0]
print("\nCoherencia:", res_coherencia/256)

# Completitud

res_completitud = 0
for i in range(0, 256):
    res_completitud += completitud(i,predicciones_XG_Bicis,objetivos_bicis,predicciones_alteradas_XG_Bicis)
print("\nCompletitud:", res_completitud/256)

# Congruencia

print ("\nCongruencia:", congruencia(predicciones_XG_Bicis, objetivos_bicis,predicciones_alteradas_XG_Bicis)[0])


Identidad: True

Separabilidad: True

Selectividad: 97.9385685232298

Coherencia: 18.361757706850767

Completitud: -0.4271963285282254

Congruencia: 44.496502


In [541]:
# Pen Random Forest

# Identidad

res_identidad = True
for i in range(0, 256):
    res_identidad &= identidad(i,atributos_pruebaPen[i],explicaciones_RF_Pen)
print("\nIdentidad:", res_identidad)

# Separabilidad

res_separabilidad = True
for i in range(0, 255):
    res_separabilidad &= separabilidad(i, 255,atributos_pruebaPen[i],atributos_pruebaPen[255],explicaciones_RF_Pen)
print("\nSeparabilidad:", res_separabilidad)

# Estabilidad

res_estabilidad = True
for i in range(0, 256):
    res_estabilidad &= estabilidad(i,atributos_pruebaPen[i],atributos_pruebaPen[:256],explicaciones_RF_Pen)
print("\nEstabilidad:", res_estabilidad)

# Selectividad

print("\nSelectividad:", selectividad(atributos_pruebaPen,5,modelRandomForestPen,objetivos_pen,"clasificacion"))

# Coherencia

res_coherencia = 0
for i in range(0, 256):
    res_coherencia += coherencia(i,predicciones_RF_Pen,objetivos_pen,predicciones_alteradas_RF_Pen)[0]
print("\nCoherencia:", res_coherencia/256)
    
# Completitud

res_completitud = 0
for i in range(0, 256):
    res_completitud += completitud(i,predicciones_RF_Pen,objetivos_pen,predicciones_alteradas_RF_Pen)
print("\nCompletitud:", res_completitud/256)

# Congruencia

print ("\nCongruencia:", congruencia(predicciones_RF_Pen, objetivos_pen,predicciones_alteradas_RF_Pen)[0])


Identidad: True

Separabilidad: True

Estabilidad: False

Selectividad: 0.9995343706392157

Coherencia: 0.86328125

Completitud: 0.0015143473518532083

Congruencia: 1.9143933068725552


In [542]:
# Pen XGBoost

# Identidad

res_identidad = True
for i in range(0, 256):
    res_identidad &= identidad(i,dtestPen.slice([i]).get_data().data,explicaciones_RF_Bicis)
print("\nIdentidad:", res_identidad)

# Separabilidad

res_separabilidad = True
for i in range(0, 255):
    res_separabilidad &= separabilidad(i, 255,dtestPen.slice([i]).get_data().data,dtestPen.slice([255]).get_data().data,explicaciones_RF_Bicis)
print("\nSeparabilidad:", res_separabilidad)

# Selectividad

print("\nSelectividad:", selectividad(dtestPen,5,modelXGBoostPen,objetivos_pen,"xgboost"))

# Coherencia

res_coherencia = 0
for i in range(0, 256):
    res_coherencia += coherencia(i,predicciones_XG_Pen,objetivos_pen,predicciones_alteradas_XG_Pen)[0]
print("\nCoherencia:", res_coherencia/256)
    
# Completitud

res_completitud = 0
for i in range(0, 256):
    res_completitud += completitud(i,predicciones_XG_Pen,objetivos_pen,predicciones_alteradas_XG_Pen)
print("\nCompletitud", res_completitud/256)

# Congruencia

print ("\nCongruencia:", congruencia(predicciones_XG_Pen, objetivos_pen,predicciones_alteradas_XG_Pen)[0])


Identidad: True

Separabilidad: True

Selectividad: 0.875

Coherencia: 1.40625

Completitud 0.002826303798428853

Congruencia: 2.146718
